In [60]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from config import api_key
from pprint import pprint

In [61]:
master_data = '../Data/clean/master_data.csv'

In [62]:
master_data_df = pd.read_csv(master_data)

In [63]:
master_data_df.head()


,Unnamed: 0,City,State,lat,long,Metro,County,2010-12,2011-12,2012-12,...,2016-12,2017-12,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019_Avg,State_abbrev
0,0,Round Rock,Texas,30.5252,-97.6660,Austin-Round Rock,Williamson County,NaN,794.5,805.0,...,884.0,991.0,1063.5,1042.5,1015.0,1000.0,1020.0,1050.0,1025.5,TX
1,1,Tampa,Florida,27.9701,-82.4797,Tampa-St. Petersburg-Clearwater,Hillsborough County,NaN,NaN,NaN,...,1220.0,1381.0,1476.5,1432.0,1440.0,1435.0,1480.0,1499.0,1457.2,FL
2,2,Scottsdale,Arizona,33.6843,-111.8611,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,NaN,NaN,...,1179.5,1268.0,1429.0,1418.5,1406.0,1365.0,1479.0,1450.0,1423.7,AZ
3,3,Oakland,California,37.7698,-122.2257,San Francisco-Oakland-Hayward,Alameda County,NaN,NaN,NaN,...,2000.0,2200.0,2997.5,2910.0,3050.0,3100.0,3055.0,3062.5,3035.5,CA
4,4,Albuquerque,New Mexico,35.1056,-106.6474,Albuquerque,Bernalillo County,NaN,NaN,625.0,...,646.0,675.0,764.0,772.5,765.0,769.0,775.0,775.0,771.3,NM


In [64]:
# weather 
# snow fall
# create a flag 0 to 1 0 doesn't snow 1 it snows 
# freezing cities 

In [65]:
units = "metric"
url = "http://api.openweathermap.org/data/2.5/weather?"

In [89]:
count = 0
humidity = []
city_name = []
max_temp = []
min_temp = []
weather = []
for city in master_data_df["City"]:
    city = city.replace(' ','%20')
    query_url = url + "appid=" + api_key + "&q=" + city + "&units=" + units
    try:
        json_weather = requests.get(query_url).json()
        humidity.append(json_weather["main"]["humidity"])
        max_temp.append(json_weather["main"]["temp_max"])
        min_temp.append(json_weather["main"]["temp_min"])
        weather.append(json_weather["weather"][0]["main"])
        city = city.replace('%20',' ')
        city_name.append(city)
        if count >= 5:
            break
        count+= 1
    except:
        BOLD = '\033[1m'
        END = '\033[0m'
        city = city.replace('%20',' ')
        print(f"{city.title()} was {BOLD}NOT{END} found. Skipping...")
        if count >= 5:
            break
        count+= 1  
pprint(json_weather)


{'base': 'stations',
 'clouds': {'all': 40},
 'cod': 200,
 'coord': {'lat': 39.08, 'lon': -94.56},
 'dt': 1562443827,
 'id': 4393217,
 'main': {'humidity': 51,
          'pressure': 1019,
          'temp': 30.05,
          'temp_max': 32.22,
          'temp_min': 28},
 'name': 'Kansas City',
 'sys': {'country': 'US',
         'id': 4246,
         'message': 0.0091,
         'sunrise': 1562410706,
         'sunset': 1562464028,
         'type': 1},
 'timezone': -18000,
 'visibility': 16093,
 'weather': [{'description': 'scattered clouds',
              'icon': '03d',
              'id': 802,
              'main': 'Clouds'}],
 'wind': {'deg': 337.389, 'speed': 3.93}}


In [92]:
weather_dict = {"City": city_name, "Weather": weather, "Max Temperature": max_temp, "Min Temperature": min_temp, "Humidity": humidity}

In [93]:
weather_df = pd.DataFrame(weather_dict)

In [94]:
weather_df.head()

,City,Weather,Max Temperature,Min Temperature,Humidity
0,Round Rock,Clear,35.56,32.22,55
1,Tampa,Rain,33.00,25.00,70
2,Scottsdale,Clear,40.56,36.00,8
3,Oakland,Clouds,28.89,14.44,52
4,Albuquerque,Haze,30.56,21.67,41


In [95]:
master_weather_df = pd.merge(weather_df, master_data_df, on="City", how="outer")

In [96]:
master_weather_df.head()

,City,Weather,Max Temperature,Min Temperature,Humidity,Unnamed: 0,State,lat,long,Metro,...,2016-12,2017-12,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019_Avg,State_abbrev
0,Round Rock,Clear,35.56,32.22,55.0,0,Texas,30.5252,-97.6660,Austin-Round Rock,...,884.0,991.0,1063.5,1042.5,1015.0,1000.0,1020.0,1050.0,1025.5,TX
1,Tampa,Rain,33.00,25.00,70.0,1,Florida,27.9701,-82.4797,Tampa-St. Petersburg-Clearwater,...,1220.0,1381.0,1476.5,1432.0,1440.0,1435.0,1480.0,1499.0,1457.2,FL
2,Scottsdale,Clear,40.56,36.00,8.0,2,Arizona,33.6843,-111.8611,Phoenix-Mesa-Scottsdale,...,1179.5,1268.0,1429.0,1418.5,1406.0,1365.0,1479.0,1450.0,1423.7,AZ
3,Oakland,Clouds,28.89,14.44,52.0,3,California,37.7698,-122.2257,San Francisco-Oakland-Hayward,...,2000.0,2200.0,2997.5,2910.0,3050.0,3100.0,3055.0,3062.5,3035.5,CA
4,Albuquerque,Haze,30.56,21.67,41.0,4,New Mexico,35.1056,-106.6474,Albuquerque,...,646.0,675.0,764.0,772.5,765.0,769.0,775.0,775.0,771.3,NM
